In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Dataset Mnist

In [ ]:
# Transformação: converte imagens para tensor,convertendo a imagem visual em uma matriz de números de forma que a rede neural possa usar como entrada no treinamento
transform = transforms.ToTensor()

# Carregar os dados de treino e teste
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Criar data loaders dividindo em batches de 64 fotos embaralhados
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

3. Definição da CNN

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1,10, kernel_size=5) # Primeira Camada Convulacional
        self.conv2 = nn.Conv2d(10,20, kernel_size=5) # Segunda camada Convulacional
        self.fc1 = nn.Linear(320,50) # Entrada: 320 unidades (equivalente a 20 canais × 4 × 4 após convoluções + pooling)
        self.fc2 = nn.Linear(50,10)  #última camada (sem ativação, pois usamos CrossEntropyLoss, que já aplica softmax internamente).
    
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2)) # Aplica a primeira convolução → ReLU → MaxPooling (2x2), reduzindo a dimensão espacial da imagem
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = x.view(-1,320) # Achata (flatten) o tensor para transformar em vetor 1D antes de passar pelas camadas densas.

        x = F.relu(self.fc1(x)) 
        return self.fc2(x)
    
model = SimpleCNN().to(device)

4. Treinamento do Modelo

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #Otimizador Adam com uma learning rate de 0.001
loss_fn = nn.CrossEntropyLoss() #Função de custo com softMax interna

def train_model(model, epochs=3): # Treinamendo da rede convulacional com 3 epochs
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad() #Reinicializaos gradientes acumulados nos parâmetros do modelo antes de calcular novos gradientes na próxima época.
            outputs = model(images) # O modelo pega as imagens de entrada, passa por todas as camadas (conv, pooling, fully connected) e devolve os valores brutos das predições para cada classe (0 a 9 no caso do MNIST).
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} finalizada.")

train_model(model)


5. Avaliação no Conjunto de teste

In [ ]:
def test_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total


acc_original = test_accuracy(model, test_loader)
print(f"Acurácia original: {acc_original * 100:.2f}%")

6. Função de ataque FGSM

In [ ]:
def fgsm_attack(image,label,epsilon):
    image.requires_grad = True
    output = model(image)
    loss = loss_fn(output,label)
    model.zero_grad()
    loss.backward()
    data_grad = image.grad.data
    perturbed_image = image + epsilon * data_grad.sign()
    return torch.clamp(perturbed_image,0,1) #Ela limita (clampa) os valores do tensor entre 0 e 1.



7. Visualizar Ataque em uma imagem

In [ ]:
dataiter = iter(test_loader)
image, label = next(dataiter)
image, label = image.to(device), label.to(device)

# Previsão original
output = model(image)
_, pred_orig = output.max(1)

# Aplicar ataque FGSM
epsilon = 0.25 #Aplicação da distorção
image_adv = fgsm_attack(image, label, epsilon)

# Previsão após o ataque
output_adv = model(image_adv)
_, pred_adv = output_adv.max(1)

# Mostrar imagens lado a lado
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.title(f"Original: {pred_orig.item()}")
plt.imshow(image.squeeze().detach().cpu().numpy(), cmap="gray")

plt.subplot(1,2,2)
plt.title(f"Adversarial: {pred_adv.item()}")
plt.imshow(image_adv.squeeze().detach().cpu().numpy(), cmap="gray")

plt.suptitle("Exemplo de Ataque FGSM")
plt.show()


Avaliar Modelo com dados Adversariais

In [ ]:
def test_adversarial(model, loader, epsilon):
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        adv_images = fgsm_attack(images, labels, epsilon)
        outputs = model(adv_images)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    return correct / total

acc_adv = test_adversarial(model, test_loader, epsilon)
print(f"Acurácia após ataque adversarial (ε={epsilon}): {acc_adv * 100:.2f}%")
